In [2]:
print("test")
api_key ="sk-957507351f464e5ab361ed598d5e904d"
SAMPLE_MD = """
# 登录与用户管理模块需求说明

## 1. 用户登录功能
用户可以在登录页面输入手机号与密码进行登录。
- 手机号必须为 11 位数字。
- 密码需至少 8 位，并且包含数字与字母。
- 若密码错误次数超过 5 次，则账号锁定 10 分钟。

请求方式：
POST /api/login

请求参数：
| 参数 | 类型 | 说明 |
|------|------|------|
| phone | string | 手机号 |
| password | string | 登录密码 |

响应示例（成功）：
{
  "code": 0,
  "msg": "ok",
  "data": { "token": "xxxxx" }
}

响应示例（失败）：
{
  "code": 401,
  "msg": "Unauthorized"
}

## 2. 用户信息查询
用户登录后可以查询个人资料。

请求方式：
GET /api/user/info

该接口需要携带 token。若 token 无效，则返回 403。
"""


test


In [3]:
system_prompt= """
#角色
你是软件需求分析专家。

#职责
请从用户提供的需求文档中，**识别所有独立的功能点（feature）**，并对每个功能点分别抽取以下字段，输出为一个由多个对象组成的数组。
对于每个 feature，请提取：
  - feature : 短标题，不超过8个汉字
  - description : 一段简洁描述
  - inputs : 输入参数名。如果有多个参数，请用逗号分隔。
  - outputs : 输出字段名。如果有多个参数，请用逗号分隔。
  - rules : 业务规则或约束（句子）。如果有多个参数，请用逗号分隔。
  - exceptions : 异常场景的简短描述。如果有多个参数，请用逗号分隔。
  
#输出要求
1. 如果无法提取某字段，请返回空数组或空字符串。
2. 对规则进行简洁化，不要加入实现细节。
3. 最终输出为一个 JSON 数组，每个 feature 为一个对象。每个 feature 对象格式如下：
{{
  "feature": "简短标题",
  "description": "一段简洁描述",
  "inputs": ["输入参数名1", "输入参数名2", ...],
  "outputs": ["输出字段名1", "输出字段名2", ...],
  "rules": ["业务规则或约束1", "业务规则或约束2", ...],
  "exceptions": ["异常场景1", "异常场景2", ...]
}}
4.不要遗漏任何可能的功能点。

# 用户输入的需求文档片段如下：
{fragment}
"""

user_prompt = """我的需求文档如下：
{fragment}
"""

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
url = 'https://api.deepseek.com/v1'
client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)
system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
messages = chat_prompt.format_messages(fragment=SAMPLE_MD)
resp = client.invoke(messages)
print(resp.content)

/var/folders/34/z5hng4mx7kb6jkvh_8m_6cph0000gn/T/ipykernel_34636/3021834747.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)


```json
[
  {
    "feature": "用户登录",
    "description": "用户通过手机号和密码进行身份验证，验证成功后返回访问令牌。",
    "inputs": ["phone", "password"],
    "outputs": ["code", "msg", "data.token"],
    "rules": ["手机号必须为11位数字", "密码需至少8位，且包含数字与字母", "若密码错误次数超过5次，则账号锁定10分钟"],
    "exceptions": ["手机号格式错误", "密码不符合规则", "账号被锁定", "用户名或密码错误"]
  },
  {
    "feature": "查询用户信息",
    "description": "已登录用户查询自己的个人资料信息。",
    "inputs": ["token"],
    "outputs": ["code", "msg", "data"],
    "rules": ["请求必须携带有效的访问令牌"],
    "exceptions": ["访问令牌无效或缺失"]
  }
]
```


In [ ]:
api_system_prompt = """
请忽略文档结构，不进行语义理解，  
而是对全文进行逐行扫描，并按照如下规则强制提取全部 API：

【强制提取规则】
1. 只要一行中同时出现：
   - HTTP 方法（GET|POST|PUT|DELETE）
   - 以 / 开头的 URL（例如 /api/...）
   就必须识别为一个 API，立即记录下来。
   不允许遗漏。

2. 对每个匹配到的 API，输出以下结构：
   - 方法
   - URL
   - 若该 API 所在章节标题能推断名称，则填写名称
   - 紧随其后的描述文字归入“说明”
   - 该章节中出现的参数表收集到“请求参数”
   - 该章节中的响应示例全部列出

3. 不允许模型因为“文档位置不常见”而忽略任何 API。
4. 不需要推断不存在的 API，只提取文档里明确写出的。

正则匹配模式参考：
- 方法：(?i)\b(GET|POST|PUT|DELETE)\b
- URL：\/[A-Za-z0-9_\/\-]+
"""
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
url = 'https://api.deepseek.com/v1'
client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)
system_prompt = SystemMessagePromptTemplate.from_template(api_system_prompt)
user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
messages = chat_prompt.format_messages(fragment=SAMPLE_MD)
resp = client(messages)
print(resp.content)

In [ ]:
flow_prompt = """你是一个流程提取代理。从给定片段中提取任何用户/流程流，并以 JSON 数组形式返回，每个流程格式如下：
{{
 "name": "简短名称",
 "steps": ["步骤1", "步骤2", ...]
}}
如果没有流程，返回 []。
只返回 JSON。
"""

user_flow_prompt = """我的需求文档如下：
{fragment}
"""
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
url = 'https://api.deepseek.com/v1'
client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)
system_prompt = SystemMessagePromptTemplate.from_template(flow_prompt)
user_prompt = HumanMessagePromptTemplate.from_template(user_flow_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
messages = chat_prompt.format_messages(fragment=SAMPLE_MD)
resp = client(messages)
print(resp.content)

In [7]:
rule_prompt = """从以下文本片段中提取明确的规则和约束，并输出为 JSON 数组：
[
  {{"text": "原始句子", "type": "input_validation|rate_limit|auth|other", "normalized": ""}}
]
只返回 JSON。
"""

user_rule_prompt = """我的需求文档如下：
{fragment}
"""
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
url = 'https://api.deepseek.com/v1'
client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)
system_prompt = SystemMessagePromptTemplate.from_template(rule_prompt)
user_prompt = HumanMessagePromptTemplate.from_template(user_rule_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
messages = chat_prompt.format_messages(fragment=SAMPLE_MD)
resp = client(messages)
print(resp.content)

[
  {"text": "手机号必须为 11 位数字。", "type": "input_validation", "normalized": "手机号长度必须为11位，且仅包含数字。"},
  {"text": "密码需至少 8 位，并且包含数字与字母。", "type": "input_validation", "normalized": "密码长度至少为8个字符，且必须同时包含数字和字母。"},
  {"text": "若密码错误次数超过 5 次，则账号锁定 10 分钟。", "type": "rate_limit", "normalized": "连续登录失败次数达到5次后，账号将被锁定10分钟。"},
  {"text": "该接口需要携带 token。若 token 无效，则返回 403。", "type": "auth", "normalized": "访问该接口必须在请求中提供有效的认证令牌（token），否则返回HTTP状态码403（禁止访问）。"}
]


In [7]:
trm_result = {
    "features": [
        {
            "id": "F_4aa9ded3",
            "name": "用户登录",
            "description": "用户通过手机号和密码进行登录验证，成功后返回访问令牌。"
        }
    ],
    "flows": [
        {
            "feature_id": "F_4aa9ded3",
            "flow_id": "FLOW_09337a5b",
            "name": "用户登录",
            "steps": [
                "访问登录页面",
                "输入11位手机号",
                "输入至少8位且包含数字与字母的密码",
                "提交登录请求",
                "系统验证并返回结果（成功则返回token，失败则可能锁定账号）"
            ]
        }
    ],
    "api": [
        {
            "方法": "POST",
            "URL": "/api/login",
            "名称": "用户登录功能",
            "说明": "用户可以在登录页面输入手机号与密码进行登录。\n- 手机号必须为 11 位数字。\n- 密码需至少 8 位，并且包含数字与字母。\n- 若密码错误 次数超过 5 次，则账号锁定 10 分钟。",
            "请求参数": [
                {
                    "参数": "phone",
                    "类型": "string",
                    "说明": "手机号"
                },
                {
                    "参数": "password",
                    "类型": "string",
                    "说明": "登录密码"
                }
            ],
            "响应示例": [
                {
                    "code": 0,
                    "msg": "ok",
                    "data": {
                        "token": "xxxxx"
                    }
                },
                {
                    "code": 401,
                    "msg": "Unauthorized"
                }
            ],
            "feature_id": ""
        }
    ],
    "rules": [
        {
            "feature_id": "F_4aa9ded3",
            "rule_id": "R_6e405da2",
            "text": "手机号必须为 11 位数字。",
            "type": "input_validation",
            "normalized": "手机号格式必须为11位数字。"
        },
        {
            "feature_id": "F_4aa9ded3",
            "rule_id": "R_8b9ef608",
            "text": "密码需至少 8 位，并且包含数字与字母。",
            "type": "input_validation",
            "normalized": "密码长度至少8位，且必须包含数字和字母。"
        },
        {
            "feature_id": "F_4aa9ded3",
            "rule_id": "R_cd8c9fe7",
            "text": "若密码错误次数超过 5 次，则账号锁定 10 分钟。",
            "type": "rate_limit",
            "normalized": "连续密码错误超过5次，账号将被锁定10分钟。"
        }
    ],
    "exceptions": [],
    "test_requirements": []
}

In [17]:
system_prompt= """你是一个资深软件测试架构师，你的角色是：

“TRM → 测试用例 转换专家”。

你精通传统与现代测试设计方法，包括：
- 等价类划分
- 边界值分析
- 场景法（Scenario-based）
- 状态机/流程测试
- 业务规则测试
- 异常/负向测试设计
- API 参数组合测试

你的任务非常明确：
基于输入的 TRM（测试需求模型），生成完整、结构化、高可执行性的测试用例集。

要求：
- 不得臆造需求，所有测试用例必须严格来源于 TRM
- 不得遗漏 TRM 中的任何 Feature、Flow、API、Rule、Exception
- 输出必须结构化且可被程序解析
- 输出必须可用于真实测试工程

请按照以下规则将 TRM（Test Requirement Model）转换为测试用例：

===============================
【测试用例覆盖要求】
===============================
1. Feature（功能点）
   - 每一个 Feature 必须生成一个 Test Suite（测试套件）

2. Flow（流程）
   - 每个流程（主流程、分支流程）必须生成至少 1 条测试用例
   - 可能需要多个：主流程、分支、异常路径

3. Rule（业务规则）
   - 对每条规则生成：
      * 等价类测试用例（有效/无效）
      * 边界值测试用例（上下限/极值）
      * 必要时生成组合测试用例

4. API（接口）
   - 每个 API 需要生成：
      * 参数合法性测试（等价类）
      * 参数边界测试
      * 参数缺失、格式错误测试
      * 错误码/返回体异常测试

5. Exception（异常）
   - 根据异常描述生成负向/容错类测试用例

===============================
【测试用例结构要求】
===============================
每一条测试用例必须包含以下字段：

- case_id：格式为 TC-feature_id-3位数字序号，如 TC-F1-001
- feature_id：所属功能点 ID
- title：测试用例标题
- type：用例类型
    (main_flow / sub_flow / rule_boundary / rule_equivalence / api_validation / exception / scenario)
- preconditions：前置条件（可为空）
- steps：测试步骤（步骤级）
- inputs：输入数据（结构化）
- expected_results：预期结果
- priority：优先级（P0 / P1 / P2）

===============================
【输出格式要求】
===============================
你的输出必须是纯 JSON，不得出现任何解释性文字。
格式如下：

{{
  "feature_id": "",
  "feature": "",
  "test_cases": []
}}

如果 TRM 中存在多个 Feature，则输出一个 JSON 数组。


===============================
【重要限制】
===============================
- 不允许虚构需求
- 不允许添加 TRM 中不存在的字段
- 不允许生成不确定性语言（例如“可能”、“大概”）
- 输出必须100%为 JSON 可解析格式

"""

user_prompt="""下面是从需求文档中抽取出来的 TRM（测试需求模型）。

请根据 TRM 严格生成测试用例。

TRM 内容如下：
{trm_json}

请按要求输出结构化 JSON（不要输出额外解释）。
"""

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
import json
url = 'https://api.deepseek.com/v1'
client = ChatOpenAI(api_key=api_key, base_url=url, model_name='deepseek-chat', temperature=0.9)
system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
messages = chat_prompt.format_messages(trm_json=json.dumps(trm_result))
resp = client(messages)
print(resp.content)

C:\Users\yuanyang\AppData\Local\Temp\ipykernel_28872\1514833114.py:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resp = client(messages)


[
  {
    "feature_id": "F_4aa9ded3",
    "feature": "用户登录",
    "test_cases": [
      {
        "case_id": "TC-F_4aa9ded3-001",
        "feature_id": "F_4aa9ded3",
        "title": "用户登录-主流程-成功登录",
        "type": "main_flow",
        "preconditions": "用户拥有有效的手机号和密码",
        "steps": [
          "访问登录页面",
          "输入11位手机号",
          "输入至少8位且包含数字与字母的密码",
          "提交登录请求"
        ],
        "inputs": {
          "phone": "13800138000",
          "password": "pass1234"
        },
        "expected_results": "系统验证成功并返回token",
        "priority": "P0"
      },
      {
        "case_id": "TC-F_4aa9ded3-002",
        "feature_id": "F_4aa9ded3",
        "title": "用户登录-规则-手机号边界值-10位数字",
        "type": "rule_boundary",
        "preconditions": "",
        "steps": [
          "访问登录页面",
          "输入10位数字手机号",
          "输入有效密码",
          "提交登录请求"
        ],
        "inputs": {
          "phone": "1380013800",
          "password": "pass1234"
        },
        "expected_results": "系统验证